In [1]:
import pandas as pd
from decimal import Decimal
import json
import math
import random
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-5

In this demo we investigate the **Fast Router performance** against test orders

### Current Issues
- tradeBySource orderIndex=1 example 53

Initialize a fast simulator

In [2]:
filename = '20221219_tradeBySourceAmount.json'
if 'Target' in filename:
    is_by_target = True
else:
    is_by_target = False
f = open(filename , "r")
bm = f.read()
bm_json = json.loads(bm)
print(len(bm_json))

120


In [3]:
def decimalize_dict(dicta):
    return({k:Decimal(v) for k,v in dicta.items()})

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['delta_liquidity'] = order0['newLiquidity'] - order0['liquidity']
    order0['y_int'] = calc_y_int(order0['liquidity'], order0['highestRate'], order0['lowestRate'], order0['marginalRate'])
    order0['fix_trade'] = order0['y_int'] - order0['liquidity']
    order0['y_int_minus_fixtrade'] = order0['y_int'] - order0['fix_trade']
    order0['order'] = order_count
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    for order in strategy:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    # orders['pair'] = [Sim.carbon_pair.pair_slash, Sim.carbon_pair.reverse.pair_slash]
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)


In [4]:
len(bm_json)

120

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

setup_index = 18 #random.randint(0, len(bm_json))
print(setup_index)
strats = pd.DataFrame()
strat_count = 0
order_count = 0
for strategy in bm_json[setup_index]['strategies']:
    orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
    strats = pd.concat([strats, orders])
strats

18


,liquidity,lowestRate,highestRate,marginalRate,newLiquidity,newMarginalRate,delta_liquidity,y_int,fix_trade,y_int_minus_fixtrade,order,strategy
0,802533,8.000000000000,9.000000000000,8.915813083046,793623,8.905373028022,-8910,874191.999999720242322589856311307319718033426...,71658.9999997202423225898563113073197180334260...,802533.000000000000000000000000000000000000000...,0,0
1,2951006,1.000000000000,2.000000000000,1.995275319802,2952006,1.995670278001,1000,2962962.00000021656034060421751829969653568059...,11956.0000002165603406042175182996965356805955...,2951006.00000000000000000000000000000000000000...,1,0
2,866645,9.000000000000,10.000000000000,9.858640850300,846948,9.838691375122,-19697,1005647.99999990016446891800833667808184541086...,139002.999999900164468918008336678081845410866...,866645.000000000000000000000000000000000000000...,2,1
3,3910757,2.000000000000,3.000000000000,2.988901743097,3912757,2.989458128836,2000,3950615.99999943938276914759332119005173906535...,39858.9999994393827691475933211900517390653512...,3910757.00000000000000000000000000000000000000...,3,1
4,938702,10.000000000000,11.000000000000,10.823565660820,906275,10.794572623011,-32427,1135103.99999939983863186625695836851104353731...,196401.999999399838631866256958368511043537313...,938702.000000000000000000000000000000000000000...,4,2
5,4860523,3.000000000000,4.000000000000,3.983143671371,4863523,3.983793441239,3000,4938270.00000117774706337371261346178726347760...,77747.0000011777470633737126134617872634776040...,4860523.00000000000000000000000000000000000000...,5,2
6,1024490,11.000000000000,12.000000000000,11.808110874391,977334,11.770285720480,-47156,1262560.00000021426212369845022920961601103236...,238070.000000214262123698450229209616011032364...,1024490.00000000000000000000000000000000000000...,6,3
7,5806276,4.000000000000,5.000000000000,4.978706927645,5810276,4.979418050729,4000,5925923.99999789025276139387962207742029181879...,119647.999997890252761393879622077420291818795...,5806276.00000000000000000000000000000000000000...,7,3
8,1129914,12.000000000000,13.000000000000,12.811021026205,1065976,12.764420280156,-63938,1388016.00000000629553734259877014384263818761...,258102.000000006295537342598770143842638187618...,1129914.00000000000000000000000000000000000000...,8,4
9,6753988,5.000000000000,6.000000000000,5.975889280009,6758988,5.976643939125,5000,6913577.99999815030464878575243878238486524444...,159589.999998150304648785752438782384865244448...,6753988.00000000000000000000000000000000000000...,9,4


In [6]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

for i in strats.order[::2]:
    FastSim.add_strategy(
        tkn='USDC',
        amt_sell=strats.liquidity[i],
        psell_start=strats.highestRate[i],
        psell_end=strats.lowestRate[i],
        amt_buy=strats.liquidity[i+1],
        pbuy_start=1/strats.highestRate[i+1],
        pbuy_end=1/strats.lowestRate[i+1],
        psell_marginal=strats.marginalRate[i],
        pbuy_marginal=1/strats.marginalRate[i+1],
    )
initial_orders = FastSim.state()['orders']
initial_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,876306.54645,802533.00000,USDC,False,9.00000,8.00000,8.91354,USDC per ETH,1
1,1,ETHUSDC,ETH,2965014.74646,2951006.00000,ETH,False,0.50000,1.00000,0.50139,USDC per ETH,0
2,2,ETHUSDC,USDC,1009321.88318,866645.00000,USDC,False,10.00000,9.00000,9.85544,USDC per ETH,3
3,3,ETHUSDC,ETH,3954646.68487,3910757.00000,ETH,False,0.33333,0.50000,0.33470,USDC per ETH,2
4,4,ETHUSDC,USDC,1139802.25823,938702.00000,USDC,False,11.00000,10.00000,10.82010,USDC per ETH,5
5,5,ETHUSDC,ETH,4943858.30020,4860523.00000,ETH,False,0.25000,0.33333,0.25113,USDC per ETH,4
6,6,ETHUSDC,USDC,1267759.20541,1024490.00000,USDC,False,12.00000,11.00000,11.80474,USDC per ETH,7
7,7,ETHUSDC,ETH,5932599.26541,5806276.00000,ETH,False,0.20000,0.25000,0.20090,USDC per ETH,6
8,8,ETHUSDC,USDC,1393199.39125,1129914.00000,USDC,False,13.00000,12.00000,12.80795,USDC per ETH,9
9,9,ETHUSDC,ETH,6920854.79199,6753988.00000,ETH,False,0.16667,0.20000,0.16737,USDC per ETH,8


In [7]:
FastSim.state()['trades']

""


In [8]:
bm_json[setup_index]['actions']

[{'strategyId': 0, 'orderIndex': 1, 'tokenAmount': '1000'},
 {'strategyId': 1, 'orderIndex': 1, 'tokenAmount': '2000'},
 {'strategyId': 2, 'orderIndex': 1, 'tokenAmount': '3000'},
 {'strategyId': 3, 'orderIndex': 1, 'tokenAmount': '4000'},
 {'strategyId': 4, 'orderIndex': 1, 'tokenAmount': '5000'},
 {'strategyId': 5, 'orderIndex': 1, 'tokenAmount': '6000'},
 {'strategyId': 6, 'orderIndex': 1, 'tokenAmount': '7000'},
 {'strategyId': 7, 'orderIndex': 1, 'tokenAmount': '8000'},
 {'strategyId': 8, 'orderIndex': 1, 'tokenAmount': '9000'}]

In [9]:
if len(bm_json[setup_index]['actions']) == 1:
    actionList = [bm_json[setup_index]['actions']]
else:
    actionList = bm_json[setup_index]['actions']
    
print(f'is_by_target {is_by_target}')
for action in actionList:
    print(action)
    targetOrder = strats.query(f"strategy=={action['strategyId']}")
    targetOrderId = targetOrder['order'].values[0]
    targetOrderToken = FastSim.state()['orders'].query(f"id=={targetOrderId}")['tkn'].values[0]
    tokenAmount = Decimal(action['tokenAmount'])
    if not is_by_target:
        if action['orderIndex'] == 1:
            FastSim.amm_buys(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        else:    
            targetOrderId += 1
            FastSim.amm_buys(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        print(targetOrderToken, targetOrderId, tokenAmount)
    else:
        if action['orderIndex'] == 1:
            FastSim.amm_sells(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        else:
            targetOrderId += 1
            FastSim.amm_sells(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        print(FastSim.carbon_pair.other(targetOrderToken), targetOrderId, tokenAmount)
    

is_by_target False
{'strategyId': 0, 'orderIndex': 1, 'tokenAmount': '1000'}
USDC 0 1000
{'strategyId': 1, 'orderIndex': 1, 'tokenAmount': '2000'}
USDC 2 2000
{'strategyId': 2, 'orderIndex': 1, 'tokenAmount': '3000'}
USDC 4 3000
{'strategyId': 3, 'orderIndex': 1, 'tokenAmount': '4000'}
USDC 6 4000
{'strategyId': 4, 'orderIndex': 1, 'tokenAmount': '5000'}
USDC 8 5000
{'strategyId': 5, 'orderIndex': 1, 'tokenAmount': '6000'}
USDC 10 6000
{'strategyId': 6, 'orderIndex': 1, 'tokenAmount': '7000'}
USDC 12 7000
{'strategyId': 7, 'orderIndex': 1, 'tokenAmount': '8000'}
USDC 14 8000
{'strategyId': 8, 'orderIndex': 1, 'tokenAmount': '9000'}
USDC 16 9000


In [10]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,876306.54645,793624.66384,USDC,False,9.00000,8.00000,8.90313,USDC per ETH,1
1,1,ETHUSDC,ETH,2965014.74646,2952006.00000,ETH,False,0.50000,1.00000,0.50129,USDC per ETH,0
2,2,ETHUSDC,USDC,1009321.88318,846953.98864,USDC,False,10.00000,9.00000,9.83558,USDC per ETH,3
3,3,ETHUSDC,ETH,3954646.68487,3912757.00000,ETH,False,0.33333,0.50000,0.33463,USDC per ETH,2
4,4,ETHUSDC,USDC,1139802.25823,906285.00689,USDC,False,11.00000,10.00000,10.79124,USDC per ETH,5
5,5,ETHUSDC,ETH,4943858.30020,4863523.00000,ETH,False,0.25000,0.33333,0.25109,USDC per ETH,4
6,6,ETHUSDC,USDC,1267759.20541,977346.41690,USDC,False,12.00000,11.00000,11.76708,USDC per ETH,7
7,7,ETHUSDC,ETH,5932599.26541,5810276.00000,ETH,False,0.20000,0.25000,0.20087,USDC per ETH,6
8,8,ETHUSDC,USDC,1393199.39125,1065990.36177,USDC,False,13.00000,12.00000,12.76154,USDC per ETH,9
9,9,ETHUSDC,ETH,6920854.79199,6758988.00000,ETH,False,0.16667,0.20000,0.16735,USDC per ETH,8


In [11]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
compare_results = pd.concat([strats[['newLiquidity', 'newMarginalRate']].reset_index(drop=True), final_orders[['tkn', 'y', 'p_marg']].reset_index(drop=True)], axis=1)
compare_results.columns = ['json_liquidity', 'json_price', 'tkn', 'sim_liquidity', 'sim_price']
compare_results = pd.concat([compare_results.reset_index(drop=True), strats[['delta_liquidity']].reset_index(drop=True), (final_orders['y'] - initial_orders['y']).reset_index(drop=True)], axis=1)

compare_results = compare_results[['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_liquidity', 'y']]
compare_results.loc[:,'sim_price'] = [1/compare_results.sim_price[i] if i%2==1 else compare_results.sim_price[i] for i in compare_results.index]
# compare_results = compare_results.astype(float)
compare_results

,tkn,json_price,sim_price,json_liquidity,sim_liquidity,delta_liquidity,y
0,USDC,8.905373028022,8.90313,793623,793624.66384,-8910,-8908.33616
1,ETH,1.995670278001,1.99486,2952006,2952006.00000,1000,1000.00000
2,USDC,9.838691375122,9.83558,846948,846953.98864,-19697,-19691.01136
3,ETH,2.989458128836,2.98835,3912757,3912757.00000,2000,2000.00000
4,USDC,10.794572623011,10.79124,906275,906285.00689,-32427,-32416.99311
5,ETH,3.983793441239,3.98260,4863523,4863523.00000,3000,3000.00000
6,USDC,11.770285720480,11.76708,977334,977346.41690,-47156,-47143.58310
7,ETH,4.979418050729,4.97826,5810276,5810276.00000,4000,4000.00000
8,USDC,12.764420280156,12.76154,1065976,1065990.36177,-63938,-63923.63823
9,ETH,5.976643939125,5.97557,6758988,6758988.00000,5000,5000.00000
